# MoSDeF Overview

MoSDeF (Molecular Simulation Design Framework) consists of two core Python package, mBuild and Foyer, that, when combined with tools for workflow and analysis management (such as Signac and Signac-Flow) provide the means to perform complex molecular simulations in a **reproducible** manner. Reproducibility in this case is achieved by making all aspects of the simulation (system initialization, simulation execution, and analysis) scriptable, such that other researchers could execute your same scripts to achieve the same results.

In this overview, we will be focusing specifically on the tools we've developed to address the issue of system initialization, including the creation of a molecular model and the application of a force field (atom-typing and parameter assignment). The two tools contained within MoSDeF to address system initialization are:

  - **mBuild**: A hierarchical, component based molecule builder
  
  - **Foyer**: A package for atom-typing as well as applying and disseminating forcefields

This overview is designed to introduce you to these tools; however, more in-depth tutorials are also available within this repository.

------

To begin, we need to import the `mbuild` package, here using the alias `mb`. This will give us access to all of the data structures and functions within mBuild.

In [ ]:
import mbuild as mb

Now we'll load a CH2 moiety into an mBuild Compound by reading from a PDB structure file (created using [Avogadro](https://avogadro.cc/)). This will create an mBuild `Compound` containing three atoms (C, H, H), as well as two C-H bonds.

In [ ]:
ch2 = mb.load('utils/ch2.pdb')

mBuild has a built-in `Compound.visualize` method for use in Jupyter notebooks.

In [ ]:
ch2.visualize()

One of the methods of mBuild `Compounds` is `particles()`, which is a generator that returns all of the `Particles` within a `Compound`.

In [ ]:
list(ch2.particles())

It would be cumbersome to have to create `Compounds` by hand each time we wanted to use them. Instead, mBuild is intended to be used in such a way that reusable classes are defined for various `Compounds`. Here, we'll define a class for our CH2 moiety.

In addition, we'll add two `Ports` to the carbon atom. `Ports` in the most general sense define a location in space; however, in practice these are generally treated as dangling bonds. Here, we are saying that there are two locations on our carbon where we can attach (bond) to other particles.

In [ ]:
class CH2(mb.Compound):
    def __init__(self):
        super(CH2, self).__init__()
        mb.load('utils/ch2.pdb', compound=self)
        carbon = list(self.particles_by_name('C'))[0]
        self.add(mb.Port(anchor=carbon, orientation=[0, 1, 0], separation=0.075), 'up')
        self.add(mb.Port(anchor=carbon, orientation=[0, -1, 0], separation=0.075), 'down')

If we instantiate this class and visualize we should see the same result we obtained earlier.

In [ ]:
ch2 = CH2()
ch2.visualize(show_ports=True)

Now that we have a class defined for our CH2 moiety, we can utilize this to create an alkane chain. Here we'll define a class for an alkane chain that can take `chain_length` as an argument. We will achieve this by successively adding CH2 moieties, and capping the first and last moieties with hydrogen atoms.

In [ ]:
from mbuild.lib.atoms import H

class Alkane(mb.Compound):
    def __init__(self, chain_length=1):
        super(Alkane, self).__init__()
        last_monomer = CH2()
        hydrogen = H()
        mb.force_overlap(move_this=hydrogen,
                         from_positions=hydrogen['up'],
                         to_positions=last_monomer['up'])
        self.add(last_monomer)
        self.add(hydrogen)
        for _ in range(chain_length-1):
            current_monomer = CH2()
            mb.force_overlap(move_this=current_monomer,
                             from_positions=current_monomer['up'],
                             to_positions=last_monomer['down'])
            self.add(current_monomer)
            last_monomer=current_monomer
        hydrogen = H()
        mb.force_overlap(move_this=hydrogen,
                         from_positions=hydrogen['up'],
                         to_positions=last_monomer['down'])
        self.add(hydrogen)

If you are unfamiliar with Python, the above code may seem a bit intimidating for creating a simple linear alkane. However, the syntax is still fairly succinct _and_ once this class is defined we never have to define it again! This means that if Researcher A at University Y creates a class to set up some complex molecular system, Researcher B at University Z can simply import this class and use it. We are currently at work towards the implementation of a plugin architecture for mBuild that will make it easier for researchers to disseminate the `Compound` classes they've created.

For example, an `Alkane` class already exists within the `utils` directory of this repository. We can import that class and in two lines of code (excluding the visualization) we'll be able to construct a linear alkane of arbitrary length.

In [ ]:
from utils import Alkane
alkane = Alkane(chain_length=4)
alkane.visualize()

One immediate concern you may have is that many of these molecules we've generated feature non-realistic configurations (such as the alkane above with all angles at 180 degrees). This could be resolved through an energy minimization of the system you create in whichever simulation package you like to use. However, mBuild also offers a built-in energy minimization routine that will attempt to optimize a `Compound`'s geometry.

In [ ]:
alkane.energy_minimize(steps=5000)

In [ ]:
alkane.visualize()

Particles can be easily added and removed in mBuild. Here, we'll explore this functionality by changing our hexane molecule into _hexanol_.

First, we'll create a class for a hydroxyl group.

In [ ]:
class OH(mb.Compound):
    def __init__(self):
        super(OH, self).__init__()
        self.add(mb.Particle(name='O', pos=[0.0, 0.0, 0.0]), label='O')
        self.add(mb.Particle(name='H', pos=[0.0, 0.1, 0.0]), label='H')
        self.add_bond((self['O'], self['H']))
        self.add(mb.Port(anchor=self['O'], orientation=[0, -1, 0], separation=0.075), label='down')

Let's visualize to see if everything looks good. Note the `Port` that we've added to the oxygen.

In [ ]:
hydroxyl = OH()
hydroxyl.visualize(show_ports=True)

Now we'll remove a hydrogen from one end of our hexane and add a hydroxyl.

Below, we remove the hydrogen compound that has the label `up-cap`.
We see that there is a single port available.

In [ ]:
alkane.remove(alkane['up_cap'])
ports = alkane.all_ports()
ports

Next, we force the single available port to overlap with the `down` port of the hydroxyl compound.

In [ ]:
mb.force_overlap(move_this=hydroxyl,
                 from_positions=hydroxyl['down'],
                 to_positions=ports[0])
hexanol = mb.Compound()
hexanol.add(alkane, label='alkane')
hexanol.add(hydroxyl, label='hydroxyl')

We can now visualize to make sure our hexanol was successfully created.

In [ ]:
hexanol.visualize()

Running simulations of a single small molecule would likely not be very interesting. mBuild offers several routines to help create more complex systems. One of these routines is the `fill_box` function, which can be used to fill a box of a user-defined size with a specified number of molecules. We'll check this out now by placing 10 hexanols in a 3nm x 3nm x 3nm box.

In [ ]:
box = mb.fill_box(hexanol, n_compounds=10, box=[3, 3, 3], seed=2)
box.visualize()

Now if we wanted to actually run a simulation of this system we would need to apply a force field and write the necessary data files. mBuild handles all of this through a single `save` command, where we can pass as arguments the name of the force field to apply (which uses Foyer under the hood) and the name of the file to create, which will be formatted based on the extension. Here we will save in Gromacs `TOP` and `GRO` formats.

In [ ]:
box.save('system.top', forcefield_name='oplsaa', overwrite=True)
box.save('system.gro', overwrite=True)

The last bit of functionality we will observe is attaching molecules to a surface. In our case here, we first need to remove the hydrogen atom opposite of the hydroxyl so that we have an available port on our hexanol to attach to a surface.

In [ ]:
hexanol.remove(hexanol['alkane']['down_cap'])
hexanol.add(hexanol.all_ports()[0], 'down', containment=False)
hexanol.visualize(show_ports=True)

The surface we will attach our hexanes to is crystalline silica, which we will import from mBuild's compound library. In many cases it is desirable to expand `Compounds` with periodicity, like surfaces, in one or more dimensions. mBuild features a built-in routine, `TiledCompound` to handle this functionality. Here we will use `TiledCompound` to expand our surface in the *x* dimension.

In [ ]:
from mbuild.lib.surfaces import Betacristobalite
surface = Betacristobalite()
tiled_surface = mb.TiledCompound(surface, n_tiles=(2, 1, 1))

mBuild contains a `Pattern` class which can be used to create patterns to place molecules at desired locations in space. This is useful for many cases, including defining the arrangement of chains on a surface. Here, we will create a `Random2DPattern` of 10 points in *xy* space.

The `Pattern.apply_to_compound` method can be used to attach `Compounds` to another `Compound` (e.g. a surface) by finding the vacant ports in the `host` `Compounds` closest to those defined by the pattern. Additionally, a `backfill` can be defined that will fill any leftover ports. Here, we will define a hydrogen atom as our backfill.

In [ ]:
pattern = mb.Random2DPattern(10)
hydrogen = H()
chains, backfills = pattern.apply_to_compound(guest=hexanol, host=tiled_surface, backfill=hydrogen)

Finally, we'll add our surface, chains, and backfills to a parent `Compound` and visualize.

In [ ]:
functionalized_surface = mb.Compound()
for part in [tiled_surface, chains, backfills]:
    functionalized_surface.add(part)
functionalized_surface.visualize()